# Neutronics simulation with CAD geometry

This example creates a CAD geometry and then carries out a neutronics simulation with a heating cell tally.

This section makes a few components and places them in a reactor object. These will form the CAD model with which we will perform a neutronics simulation.

In [ ]:
import paramak

plasma = paramak.Plasma(
    minor_radius=150.,
    major_radius=450.,
    triangularity=0.55,
    elongation=2.,
    rotation_angle=180,
    name='plasma'
)

blanket = paramak.BlanketFP(
    plasma=plasma,
    thickness=50,
    stop_angle=90,
    start_angle=-90,
    offset_from_plasma=40,
    rotation_angle=180,
    name='blanket'
)

my_reactor = paramak.Reactor([plasma, blanket])
my_reactor.show()

This next code block exports the 3D reactor geometry as a DAGMC compatibile h5m file which can be used as a neutronics geometry. DAGMC is a neutronics code that allows particle transport on CAD geometry with various neutronics transport codes (including OpenMC).

More details on DAGMC here https://svalinn.github.io/DAGMC/

This export will take a reasonable amount of time compared to other cells

In [ ]:
!rm dagmc.h5m
my_reactor.export_dagmc_h5m(filename='dagmc.h5m')

The model can now be simulated in OpenMC. OpenMC is imported along with some convenience packages

In [ ]:
import openmc
import openmc_dagmc_wrapper as odw
import openmc_plasma_source as ops

This section forms the neutronics model by combining the DAGMC model with a plasma source and some assigned materials. Additionally, the tallies to record the heating are specified. The code block also sets simulation intensity and specifies the neutronics results to record (know as tallies).

In [ ]:
# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package

geometry = odw.Geometry(h5m_filename='dagmc.h5m', reflective_angles=(0, 180))

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict={
        'mat_blanket':'Li4SiO4',
        'mat_plasma': 'DT_plasma'
    }
)

tally1 = odw.CellTally(
    tally_type="heating",
    target="blanket",
    materials=materials
)

tallies = openmc.Tallies([tally1])

# odw.FusionSettings is the same as openmc.settings but it just sets the mode to fixed source and inactive batches to zero
settings = odw.FusionSettings()
settings.batches = 1
settings.particles = 100

# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package
settings.source = ops.FusionRingSource(fuel="DT", radius=350)

my_model = openmc.Model(
    materials=materials, geometry=geometry, settings=settings, tallies=tallies
)

# delete old files just in case they are there
!rm summary.h5
!rm statepoint*.h5

# starts the simulation and gets the filename of the output file produced
statepoint_file = my_model.run()

Extracting the resulting tally value from the statepoint file

In [ ]:
sp = openmc.StatePoint(statepoint_file)

heating_tally = sp.get_tally(name='blanket_heating')

# this returns the tally in unknown base units
heating_tally.get_values().sum()

This next cell just imports a post processing package and uses that package to find the units of the tally

In [ ]:
import openmc_tally_unit_converter as otuc

tally = otuc.process_tally(
    heating_tally,
)
# this prints the tally with base units identified
print(tally)

Converts the units from the base units (eV/simulated particle) to MeV/simulated particle

In [ ]:
otuc.process_tally(
    heating_tally,
    required_units="MeV/source_particle"
)

Converts units by normalising by the source strength (number of nuetrons per second)

In [ ]:
otuc.process_tally(
    heating_tally,
    required_units="gigawatts",
    source_strength=1e21
)

**Learning Outcomes from Task 11**

- Neutronics simulations can be performed by combining CAD models, neutron sources and material definitions.
- Scaled the units of the output results using post processing package